In [ ]:
import matplotlib.pyplot as plt
import glob,scipy
from astropy.io import fits
import numpy as np
import scipy.ndimage as snd
from scipy import optimize
#import seaborn as sb

%pylab inline

# First grab a list of all filenames inside of the defined root directory

In [ ]:
rootdir='hutch_data_files/'
all_fits_filenames=np.array(glob.glob(rootdir+'/*/*.fit'))
folder_names=np.array([fooname.split('/')[-2] for fooname in all_fits_filenames])
print(all_fits_filenames[:5],"...",all_fits_filenames[-5:])
print(unique(folder_names))

### Now, using the image headers read in from pyfits.getheader, sort the exposures into flats, biases, science frames, etc., as well as reading in the filter names and exposure times

In [ ]:
all_exp_types = np.array([fits.getheader(fooname)["IMAGETYP"] for fooname in all_fits_filenames])
print("Our exposure types",unique(all_exp_types))

bias_filenames = all_fits_filenames[np.where(all_exp_types=='Bias Frame')]
print("Number of bias frames",len(bias_filenames))

dark_filenames = np.sort(all_fits_filenames[np.where(all_exp_types=='Dark Frame')])
dark_exptimes=np.array([fits.getheader(fooname)["EXPTIME"] for fooname in dark_filenames])
print("Number of darks and exposure times",str(len(dark_filenames)),unique(dark_exptimes))

flat_filenames = all_fits_filenames[np.where(all_exp_types=='Flat Field')]
flat_filter_names=np.array([fits.getheader(fooname)["FILTER"] for fooname in flat_filenames])
flat_exptimes=np.array([fits.getheader(fooname)["EXPTIME"] for fooname in flat_filenames])
print("Flat filters and exposure times",unique(flat_filter_names),unique(flat_exptimes))

object_folder_name='m101'
object_filenames= all_fits_filenames[np.where((folder_names==object_folder_name))]

object_filter_names=np.array([fits.getheader(fooname)["FILTER"] for fooname in object_filenames])
object_exptimes=np.array([fits.getheader(fooname)["EXPTIME"] for fooname in object_filenames])
print("Object filters exposed and exposure times",unique(object_filter_names),unique(object_exptimes))


### Let's make a median bias, flat, and dark frame. Use the definition below to stack them up and take the median value for each pixel in the stack

In [ ]:
# Warning, reading in hundreds of bias frames may slow/kill your computer!
# Index the filename array if you want to use a subset.

def median_combine(filelist):
    allimgs=[]
    for filename in filelist: allimgs.append(fits.getdata(filename))
    allimgs=np.array(allimgs)
    medianimg=np.median(allimgs,axis=0)
    return medianimg


In [ ]:
# Make a median bias frame and save it to the "calib" directory

caldir = rootdir+'calib/'

median_bias = median_combine(bias_filenames)

fits.writeto(caldir+'median_bias.fits', median_bias, clobber=True)

In [ ]:
# Make a median dark frame and save it to the "calib" directory

# 60 second dark frames
dark_60 = np.where(dark_exptimes==60)
print(dark_filenames[dark_60])

median_dark_60 = median_combine(dark_filenames[dark_60])

fits.writeto(caldir+'median_dark_60s.fits', median_dark_60, clobber=True)

In [ ]:
flat_filter_names,flat_exptimes[np.where(flat_filter_names=='H-a')]

In [ ]:
# Make median flat frames for each filter

# B band
thefilt = 'B'
flat_time = 30
flat_files = flat_filenames[np.where((flat_filter_names==thefilt) & (flat_exptimes==flat_time))]
print(flat_files)

median_flat=median_combine(flat_files)
fits.writeto(caldir+'median_flat_'+thefilt+'.fits',median_flat,clobber=True)

# V band
thefilt = 'V'
flat_time = 10
flat_files = flat_filenames[np.where((flat_filter_names==thefilt) & (flat_exptimes==flat_time))]
print(flat_files)

median_flat=median_combine(flat_files)
fits.writeto(caldir+'median_flat_'+thefilt+'.fits',median_flat,clobber=True)

# R band
thefilt = 'R'
flat_time = 10
flat_files = flat_filenames[np.where((flat_filter_names==thefilt) & (flat_exptimes==flat_time))]
print(flat_files)

median_flat=median_combine(flat_files)
fits.writeto(caldir+'median_flat_'+thefilt+'.fits',median_flat,clobber=True)

# I band
thefilt = 'I'
flat_time = 10
flat_files = flat_filenames[np.where((flat_filter_names==thefilt) & (flat_exptimes==flat_time))]
print(flat_files)

median_flat=median_combine(flat_files)
fits.writeto(caldir+'median_flat_'+thefilt+'.fits',median_flat,clobber=True)

# [O III] band
thefilt = 'O-III'
flat_time = 60
flat_files = flat_filenames[np.where((flat_filter_names==thefilt) & (flat_exptimes==flat_time))]
print(flat_files)

median_flat=median_combine(flat_files)
fits.writeto(caldir+'median_flat_'+thefilt+'.fits',median_flat,clobber=True)

# H-alpha band
thefilt = 'H-a'
flat_time = 60
flat_files = flat_filenames[np.where((flat_filter_names==thefilt) & (flat_exptimes==flat_time))]
print(flat_files)

median_flat=median_combine(flat_files)
fits.writeto(caldir+'median_flat_'+thefilt+'.fits',median_flat,clobber=True)

# [S II] band
thefilt = 'S-II'
flat_time = 60
flat_files = flat_filenames[np.where((flat_filter_names==thefilt) & (flat_exptimes==flat_time))]
print(flat_files)

median_flat=median_combine(flat_files)
fits.writeto(caldir+'median_flat_'+thefilt+'.fits',median_flat,clobber=True)


### Display the median flat, bias, and dark.

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(median_flat,vmax=np.median(median_flat)+3*np.std(median_flat),cmap=plt.cm.coolwarm)
plt.title('Median flat - '+thefilt,fontsize=25)
plt.colorbar()
plt.figure(figsize=(10,10))
plt.imshow(median_bias,vmax=np.median(median_bias)+3*np.std(median_bias),cmap=plt.cm.RdBu_r)
plt.title('Median bias frame',fontsize=25)
plt.colorbar()

plt.figure(figsize=(10,10))
plt.imshow(median_dark_60,vmin=np.median(median_dark_60)-0.1*np.std(median_dark_60),vmax=np.median(median_dark_60)+0.1*np.std(median_dark_60),cmap=plt.cm.spring),colorbar()
plt.title('Median dark frame',fontsize=25)

### Now let's reduce the science frames using standard intrument signature removal. Note we subtract the dark and bias from the science, and divide by a normalized flat

In [ ]:
def reduce_raw_science_frames(science_filelist,bias_file,flat_file,flat_exptime,dark_file,dark_exptime):
    # Read the bias, flat, and dark files
    bias = fits.getdata(bias_file)
    dark = fits.getdata(dark_file)
    flat = fits.getdata(flat_file)
    
    # Normalize the flat field
    flat_darkcorr = (dark-bias) * (flat_exptime/dark_exptime)
    flat_corr = (flat-bias-flat_darkcorr)
    normed_flat = flat_corr/np.median(flat_corr)
    
    # Reduce the science frames
    allreducedimgs=[]
    for filename in science_filelist: 
        science_frame   = fits.getdata(filename)
        science_exptime = fits.getheader(filename)["EXPTIME"]
        dark_science    = (dark-bias) * (science_exptime/dark_exptime)
        reduced_frame   = (science_frame-dark_science-bias)/normed_flat
        allreducedimgs.append(reduced_frame)
    allreducedimgs=np.array(allreducedimgs)
    return allreducedimgs

### Read in the raw science frames and reduce them using the above definition

In [ ]:
object_filter_names,object_exptimes

In [ ]:
# Calibration filenames
bias_file = caldir+'median_bias.fits'
dark_file = caldir+'median_dark_60s.fits'
dark_exptime = 60.

# R band object images
thefilt = 'R'
flat_file = caldir+'median_flat_'+thefilt+'.fits'
flat_exptime = 10.
  # Object frames
science_filenames=object_filenames[np.where(object_filter_names==thefilt)]
print(science_filenames)
  # Reduce all frames 
science_reduced = reduce_raw_science_frames(science_filenames,bias_file,flat_file,flat_exptime,dark_file,dark_exptime)
print(science_filenames[0].replace('.fit','_'+thefilt+'_reduced.fits'))
  # Save the reduced frames to a new FITS file
for i in range(len(science_reduced)):
    fits.writeto(science_filenames[i].replace('.fit','_'+thefilt+'_reduced.fits'),science_reduced[i],clobber=True,header=fits.getheader(science_filenames[i]))

# H-alpha band object images
thefilt = 'H-a'
flat_file = caldir+'median_flat_'+thefilt+'.fits'
flat_exptime = 60.
  # Object frames
science_filenames=object_filenames[np.where(object_filter_names==thefilt)]
print(science_filenames)
  # Reduce all frames 
science_reduced = reduce_raw_science_frames(science_filenames,bias_file,flat_file,flat_exptime,dark_file,dark_exptime)
print(science_filenames[0].replace('.fit','_'+thefilt+'_reduced.fits'))
  # Save the reduced frames to a new FITS file
for i in range(len(science_reduced)):
    fits.writeto(science_filenames[i].replace('.fit','_'+thefilt+'_reduced.fits'),science_reduced[i],clobber=True,header=fits.getheader(science_filenames[i]))

# B band object images
thefilt = 'B'
flat_file = caldir+'median_flat_'+thefilt+'.fits'
flat_exptime = 30.
  # Object frames
science_filenames=object_filenames[np.where(object_filter_names==thefilt)]
print(science_filenames)
  # Reduce all frames 
science_reduced = reduce_raw_science_frames(science_filenames,bias_file,flat_file,flat_exptime,dark_file,dark_exptime)
print(science_filenames[0].replace('.fit','_'+thefilt+'_reduced.fits'))
  # Save the reduced frames to a new FITS file
for i in range(len(science_reduced)):
    fits.writeto(science_filenames[i].replace('.fit','_'+thefilt+'_reduced.fits'),science_reduced[i],clobber=True,header=fits.getheader(science_filenames[i]))


### What do the pixel values of the reduced frames look like? Note there is a sky level, which provides a noisy minimum to all of the reduced pixel values. Find it by taking a histogram of the pixel values and taking the pixel value which is at the maximum of the histogram

In [ ]:
plt.figure(figsize=(18,9))
skylevels=[]
for i in range(len(science_reduced)):
    numpix,aduvals=np.histogram(science_reduced[i].flatten(),bins=1000,range=[-1e3,5e3])
    skylevel=aduvals[where(numpix==numpix.max())][0]
    skylevels.append(skylevel)
    plt.plot(aduvals[:-1],numpix,label=str(science_filenames[i].split('/')[-1]))
    plt.axvline(skylevel,color='k')
plt.xlabel('pix val',fontsize=20)
plt.ylabel('number of pixels',fontsize=20)
plt.yscale('log')
plt.legend()
print(skylevels)
plt.title('Histogram of reduced '+thefilt+' frame pixel values',fontsize=20)

### Show an example of the difference between reduced and unreduced

In [ ]:
testnum=0  #the file number to show comparison plot
print(science_filenames[testnum])
nstd=0.3
unreduced=fits.getdata(science_filenames[testnum])
print(np.std(unreduced))
figure(figsize=(20,8))
plt.subplot(121)
#vmin_unred=np.median(median_bias)+skylevels[testnum]
vmin_unred=np.median(unreduced)
print(vmin_unred)
plt.imshow(unreduced,cmap=cm.Greys,vmin=vmin_unred,vmax=vmin_unred+nstd*np.std(unreduced))
plt.colorbar()
plt.subplot(122)
plt.imshow(science_reduced[testnum],cmap=cm.Greys,vmin=skylevels[testnum],vmax=skylevels[testnum]+nstd*np.std(unreduced))
plt.colorbar()
plt.suptitle('The difference between reduced and unreduced',fontsize=25)

### Let's find some objects in the field, using a simplified object & centroid finder

In [ ]:
def find_object_centroids_filterbysize(img,threshold,minsize):
    labels, num = snd.label(img > threshold, np.ones((3,3)))     # scipy labels/segments the image using a threshold
    centers = snd.center_of_mass(img, labels, range(1,num+1))    # scipy calculates the center of mass on the labeled img
    x = array(centers)[:,1]
    y = array(centers)[:,0]
    slices=snd.find_objects(labels)
    xs=np.array([objlabel[1].stop-objlabel[1].start for objlabel in slices])  # takes the min and max label slices
    ys=np.array([objlabel[0].stop-objlabel[0].start for objlabel in slices])  #  to find a rough object size

    maxsize=1025    # I hardcoded this in so that some spurious objects would be skipped. Change/delete if you like
    bigenough=np.where((xs>minsize) & (ys>minsize) & (xs<maxsize) & (ys<maxsize))
    xc,yc=x[bigenough],y[bigenough]
    xs,ys=xs[bigenough],ys[bigenough]
    
    print(str(len(xc))+' objects found')
    return xc,yc,xs,ys

### Test out the object finder on a single image

In [ ]:
imgnum=0
nstd_aboveskynoise=10
threshold=skylevels[imgnum]+nstd_aboveskynoise*np.sqrt(skylevels[imgnum])   # decide on a threshold using the sky noise
minsize=2

xfoo,yfoo,xsfoo,ysfoo=find_object_centroids_filterbysize(science_reduced[imgnum],threshold,minsize)


plt.figure(figsize=(20,20))
plt.imshow(science_reduced[imgnum],cmap=cm.Greys,vmin=0,vmax=1.5*threshold)
plt.plot(xfoo,yfoo,'rs',mfc='None',markersize=20,markeredgecolor='b',markeredgewidth=2)
axis([0,1024,0,1024])

### Just a little definition which can grab postage stamps of objects

In [ ]:
def get_stamp(img,starx,stary,ws):
    xlo,xhi,ylo,yhi=int(starx-ws),int(starx+ws),int(stary-ws),int(stary+ws)
    xmin,ymin=0,0
    xmax,ymax=np.shape(img)
    if xlo<xmin: xlo=xmin
    if xhi>xmax: xhi=xmax
    if ylo<ymin: ylo=ymin
    if yhi>ymax: yhi=ymax
    return img[ylo:yhi,xlo:xhi]

In [ ]:
len(xfoo)

In [ ]:
objnum=11
ws=30
stamp=get_stamp(science_reduced[imgnum],xfoo[objnum],yfoo[objnum],ws)
plt.imshow(stamp,interpolation='None',vmin=0,vmax=1.5*threshold)
xcfoo,ycfoo=ws+xfoo[objnum]-floor(xfoo[objnum]),ws+yfoo[objnum]-floor(yfoo[objnum])
plot(xcfoo,ycfoo,'wo')
plt.colorbar()
axis([0,ws*2,0,ws*2])
plt.title('a sample stamp')

### Now find objects in all the reduced frames, again using the sky noise from each reduced image as a threshold. Plot all the centroids detected in each image to illustrate the dithering that occurs between frames

In [ ]:
nstd_aboveskynoise=10
thresholds=skylevels+nstd_aboveskynoise*np.sqrt(skylevels)

#thresholds=[threshold]*len(science_reduced)
minsize=2
catalog={'x':[],'y':[]}
xc_all,yc_all,xs_all,ys_all=[],[],[],[]
figure(figsize=(15,6))
for i in range(len(science_reduced)):
    xfoo,yfoo,xsfoo,ysfoo=find_object_centroids_filterbysize(science_reduced[i],thresholds[i],minsize)
    xc_all.append(xfoo)
    yc_all.append(yfoo)
    xs_all.append(xsfoo)
    ys_all.append(ysfoo)
    subplot(121)
    plot(xfoo,yfoo,'o')
    subplot(122)
    plot(xfoo,yfoo,'.')
    axis([0,400,600,1000])
subplot(121)
title('all objects found in all frames')
subplot(122)
title('zoomed in to illustrate dithering')

### Now let's make a little definition to find all matching objects given two frame's object centroids. If objects are chosen carefully, this can give us a rough offset between the frames, as is shown in the histogram below

In [ ]:
def find_closest(xc0,yc0,xc1,yc1,dr):
    nobjs1=len(xc1)
    x_off,y_off=[],[]
    for i in range(nobjs1):
        poss_match=np.where(np.sqrt((xc1[i]-xc0)**2+(yc1[i]-yc0)**2)<dr)[0]  # for each object in the second catalog, 
        x_off.extend([(xc1[i]-xc0[j]) for j in poss_match])                  #  find matches within a radius dr
        y_off.extend([(yc1[i]-yc0[j]) for j in poss_match])
    x_off=np.array(x_off)
    y_off=np.array(y_off)
    
    n_xoff,xfoo=np.histogram(x_off,bins=dr*2,range=[-dr,dr])   # histogram all the offsets to find the maximum value
    n_yoff,yfoo=np.histogram(y_off,bins=dr*2,range=[-dr,dr])   #  which corresponds to the rough offset between frames
    x_peak=(xfoo[where(n_xoff==np.max(n_xoff))])[0]+(xfoo[1]-xfoo[0])/2.
    y_peak=(yfoo[where(n_yoff==np.max(n_yoff))])[0]+(yfoo[1]-yfoo[0])/2.   # (note addition of half bin width)
    return x_off,y_off,x_peak,y_peak


In [ ]:
secondnum=1   #change this to be the second frame you want to see the offsets for
dr=60
xc0,yc0,xc1,yc1=xc_all[0],yc_all[0],xc_all[secondnum],yc_all[secondnum]
xoff,yoff,xpeak,ypeak=find_closest(xc0,yc0,xc1,yc1,dr)
 
    
hist(xoff,histtype='step',label='x offset',range=[-dr,dr],bins=dr,color='r')
hist(yoff,histtype='step',label='y offset',range=[-dr,dr],bins=dr,color='b')

axvline(xpeak,color='r')
axvline(ypeak,color='b')

xlabel('Nearest object centroid offset [pix]',fontsize=20)
ylabel('Number of objects',fontsize=20)
title('X/Y shift between frames',fontsize=25)

figure(figsize=(10,10))
plot(xc0,yc0,'g.')
plot(xc1-xpeak,yc1-ypeak,'ro',mfc='None')

#axis([100,400,100,400])

### Going a little further than the simple offsets above, iterate on this process to find the overall shifts down to subpixel accuracy

In [ ]:
def find_frame_shifts(x0,y0,x1,y1,dr0,nsteps):    #like the def above, take in two sets of coordinates
    xshifts,yshifts=0,0
    drs=[dr0,20,10,5,2.5,1.25]
    for i in range(nsteps):
        dr=dr0/(1.+i)      # slowly reduce the matching radius, to get rid of outliers
        xoff,yoff,xpeak,ypeak=find_closest(x0,y0,x1-xshifts,y1-yshifts,dr)
        #print xoff,yoff
        n_xoff,xfoo=np.histogram(xoff,bins=50,range=[-dr,dr])
        n_yoff,yfoo=np.histogram(yoff,bins=50,range=[-dr,dr])
        #axvline(xpeak)
        print(xpeak)
        if i==0:
            xshift,yshift=xpeak,ypeak    # the first rough offset estimate is the peak of the offset histogram
        else:                            # successive offsets are found by taking the median value of the remainder
            xshift,yshift=np.median(xoff[np.abs(xoff)<dr/2.]),np.median(yoff[np.abs(yoff)<dr/2.])
        xshifts+=xshift        # add in each successive offset to the total
        yshifts+=yshift
        print(dr,xshifts,yshifts)
        #plot(xshifts,yshifts,'o')
    hist(xoff,histtype='step',color='r',bins=50)
    hist(yoff,histtype='step',color='b',bins=50)
    #plot(yfoo[1:],n_yoff,'b')
    axvline(xshift,color='r')
    axvline(yshift,color='b')
    return xshifts,yshifts

def find_closest(xc0,yc0,xc1,yc1,dr):
    nobjs1=len(xc1)
    x_off,y_off=[],[]
    for i in range(nobjs1):
        poss_match=np.where(np.sqrt((xc1[i]-xc0)**2+(yc1[i]-yc0)**2)<dr)[0]  # for each object in the second catalog, 
        x_off.extend([(xc1[i]-xc0[j]) for j in poss_match])                  #  find matches within a radius dr
        y_off.extend([(yc1[i]-yc0[j]) for j in poss_match])
    x_off=np.array(x_off)
    y_off=np.array(y_off)
    
    n_xoff,xfoo=np.histogram(x_off,bins=int(dr*2),range=[-dr,dr])   # histogram all the offsets to find the maximum value
    n_yoff,yfoo=np.histogram(y_off,bins=int(dr*2),range=[-dr,dr])   #  which corresponds to the rough offset between frames
    x_peak=(xfoo[where(n_xoff==np.max(n_xoff))])[0]+(xfoo[1]-xfoo[0])/2.
    y_peak=(yfoo[where(n_yoff==np.max(n_yoff))])[0]+(yfoo[1]-yfoo[0])/2.   # (note addition of half bin width)
    return x_off,y_off,x_peak,y_peak


### Perform this on the first and last images, to test the concept. The definition above also produces a histogram which can be used to ensure the algorithm is working properly

In [ ]:
firstnum,secondnum=0,1   #two different numbers corresponding to different frames
dr=60
xc0,yc0,xc1,yc1=xc_all[firstnum],yc_all[firstnum],xc_all[secondnum],yc_all[secondnum]
find_frame_shifts(xc0,yc0,xc1,yc1,dr,4)

### Now deploy the offset finder on all reduced images, relative to the first one

In [ ]:
xshifts,yshifts=np.zeros(len(science_reduced)),np.zeros(len(science_reduced))
for secondnum in range(1,len(science_reduced)):
    xshifts[secondnum],yshifts[secondnum]=find_frame_shifts(xc_all[0],yc_all[0],xc_all[secondnum],yc_all[secondnum],dr,4)

In [ ]:
xshifts,yshifts

### Use scipy's interpolation shifting algorithm to shift each reduced image by the negative of the offsets found above. This should match them all up

In [ ]:
shape(science_reduced)

In [ ]:
nzoom=4
resampled=np.zeros((len(science_reduced),1024*nzoom,1024*nzoom))
for k in range(len(science_reduced)):
    for i in range(1024):
        for j in range(1024):
            resampled[k,i*nzoom:(i+1)*nzoom,j*nzoom:(j+1)*nzoom]=science_reduced[k,i,j]
            

In [ ]:
science_shifted_resampled=np.array([snd.interpolation.shift(resampled[i],[-yshifts[i]*nzoom,-xshifts[i]*nzoom],order=3) for i in range(0,len(resampled))])


In [ ]:
fits.writeto(rootdir+thefilt+'-median-resampled.fits',np.median(science_shifted_resampled,axis=0),clobber='True')

In [ ]:
science_shifted_rescaled=np.array([snd.interpolation.shift(science_reduced[i],[-yshifts[i],-xshifts[i]],order=5)-skylevels[i] for i in range(0,len(science_reduced))])
stackedimg=np.median(science_shifted_rescaled,axis=0)

### Display the median of the stacked image to ensure it worked okay

In [ ]:
figure(figsize=(14,12))
title('Median stacked '+thefilt+' image',fontsize=10)
imshow(stackedimg,cmap=cm.Greys,vmin=0,vmax=5*np.sqrt(median(skylevels)))
colorbar()
#plt.axis([200,700,200,700])
fits.writeto(rootdir+thefilt+'-median.fits',stackedimg,clobber=True)

# Making an RGB image (aka align three images)

In [ ]:
rgbroot='hutch_data_files/m101/'
rimg=fits.getdata(rgbroot+'00000179.M 101_R_reduced.fits')
gimg=fits.getdata(rgbroot+'00000186.M 101_H-a_reduced.fits')
bimg=fits.getdata(rgbroot+'00000180.M 101_B_reduced.fits')


In [ ]:
nstd_r=20
nstd_g=30
nstd_b=12
minsize=3
skylevel_r,skylevel_g,skylevel_b=np.median(rimg[rimg>0]),np.median(gimg[gimg>0]),np.median(bimg[bimg>0])

xr,yr,xsfoo,ysfoo=find_object_centroids_filterbysize(rimg,skylevel_r+nstd_r*np.sqrt(skylevel_r),minsize)
xg,yg,xsfoo,ysfoo=find_object_centroids_filterbysize(gimg,skylevel_g+nstd_g*np.sqrt(skylevel_g),minsize)
xb,yb,xsfoo,ysfoo=find_object_centroids_filterbysize(bimg,skylevel_b+nstd_b*np.sqrt(skylevel_b),minsize)


plot(xr,yr,'ro',alpha=.4)
plot(xg,yg,'go',alpha=.4)
plot(xb,yb,'bo',alpha=.4)
plt.figure()
plot(xr,yr,'ro',alpha=.4)
plot(xg,yg,'go',alpha=.4)
plot(xb,yb,'bo',alpha=.4)
plt.axis([0,400,700,1000])

In [ ]:
dr=30
x_off_rb,y_off_rb=find_frame_shifts(xr,yr,xb,yb,dr,2)
x_off_rg,y_off_rg=find_frame_shifts(xr,yr,xg,yg,dr,2)

In [ ]:
figure(figsize=(10,10))
plot(xr,yr,'ro',alpha=.4)
plot(xg-x_off_rg,yg-y_off_rg,'go',alpha=.4)
plot(xb-x_off_rb,yb-y_off_rb,'bo',alpha=.4)
#axis([500,700,200,400])

In [ ]:
rgbroot

In [ ]:
g_shifted=snd.interpolation.shift(gimg,[-y_off_rg,-x_off_rg],order=1)
fits.writeto(rgbroot+'H-a-median-shifted.fits',g_shifted,clobber='True')

b_shifted=snd.interpolation.shift(bimg,[-y_off_rb,-x_off_rb],order=1)
fits.writeto(rgbroot+'B-median-shifted.fits',b_shifted,clobber='True')

In [ ]:
def scalesqrt(inputArray, scale_min=None, scale_max=None):
    """Performs sqrt scaling of the input numpy array."""

    imageData=numpy.array(inputArray, copy=True)

    imageData = imageData.clip(min=scale_min, max=scale_max)
    imageData = imageData - scale_min
    indices = numpy.where(imageData < 0)
    imageData[indices] = 0.0
    imageData = numpy.sqrt(imageData)
    imageData = imageData / math.sqrt(scale_max - scale_min)

    return imageData

### Read in three images for the r,g,b array. Each of these has been reduced, stacked, and shifted/rotated to be on a common coordinate system

In [ ]:
rgbroot

In [ ]:
rimg=fits.getdata(rgbroot+'00000179.M 101_R_reduced.fits')
gimg=fits.getdata(rgbroot+'H-a-median-shifted.fits')
bimg=fits.getdata(rgbroot+'B-median-shifted.fits')

rgbstack=[rimg,gimg,bimg]

### Do a rescaling of the image data for the purposes of making the RGB image (which needs values between 0 and 8 bits (0-255). I use the sqrt scaling used in the definition above, with a min and max defined by 0 and some number of std deviations above the sky noise

In [ ]:
nstd=[15,5,10]   # number of standard deviations above sky noise to call an RGB pixel=255
rgbimg=np.zeros((1024,1024,3))

for i in range(3):
    onecolor=rgbstack[i]
    colorstd,colormed=np.std(onecolor[onecolor>10]),np.median(onecolor[onecolor>10])
    scale_min,scale_max=colormed,colormed + nstd[i]*colorstd
    rgbimg[:,:,i]=scalesqrt(onecolor, scale_min=scale_min, scale_max=scale_max)
    

### Finally, display our RGB image with imshow

In [ ]:
figure(figsize=(10,10))
imshow(rgbimg,interpolation='None')
#axis([150,800,50,800])
axis('off')